In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
import random
import time
import csv
import selenium
from bs4 import BeautifulSoup

class StackOverflowLogin:
    def __init__(self):
        self.options = webdriver.ChromeOptions()
        self.options.add_argument('--ignore-certificate-errors')
        self.options.add_argument("--test-type")
        self.options.add_experimental_option("detach", True)
        self.driver = selenium.webdriver.Chrome(options=self.options, executable_path=r"C:\path\to\chromedriver.exe")


    def login(self, email, password):
        # Navigate to web page
        self.driver.get('https://stackoverflow.com/users/login?ssrc=head&returnurl=https%3a%2f%2fstackoverflow.com%2f')

        self.driver.maximize_window()

        # Wait for the accept cookies button to appear
        accept_button = WebDriverWait(self.driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'js-accept-cookies')))
        accept_button.click()

        # Find email field and enter email address
        email_field = self.driver.find_element(By.NAME, 'email')
        email_field.send_keys(email)

        # Find password field and enter password
        password_field = self.driver.find_element(By.NAME, 'password')
        password_field.send_keys(password)

        # Click login button
        login_button = self.driver.find_element(By.NAME, 'submit-button')
        login_button.click()
        self.driver.implicitly_wait(4000)
        time.sleep(5)
        
        # Scroll all the way down
        
        self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)
        
        # Enter the all questions 
        all_questions = self.driver.find_element(By.XPATH,"//*[@id='mainbar']/h2/a[1]")
        all_questions.click()
        time.sleep(5)
        
        # Enter the newest questions
        newest = self.driver.find_element(By.XPATH,"//*[@id='mainbar']/div[2]/div/div[2]/div/div[1]/a[1]/div")
        newest.click()
        time.sleep(5)
    
    def get_links_and_stats(self):
        all_links = []
        all_stats = []
        all_tags = []
        i = 0
        k = 0
        z = 0
        while i <= 1:
            # Get the HTML content of the page
            html = self.driver.page_source

            # Parse the HTML content with BeautifulSoup
            soup = BeautifulSoup(html, 'html.parser')

            # Find all elements with the "s-link" class
            links = soup.select('.s-post-summary--content-title')
            stats = soup.select('.s-post-summary--stats-item-number')
            tags = soup.select('.s-post-summary--meta-tags')
            # Append the text content of each link and stat to the lists
            for link in links:
                all_links.append(link.text.strip())
               
            for stat in stats:
                all_stats.append(stat.text.strip())
                
            # Append the text content of each tag to the list
            for tag in tags:
                all_tags.append(tag.text.strip())
            # Parse the tags and add commas between them
            tag_list = []
            for tags in all_tags:
                tag_list.append(tags.replace(' - ', ', '))
                

            # Scroll down to the bottom of the page and click on the "Next" button
            self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(3)
            next_button_xpath = f'//*[@id="mainbar"]/div[5]/a[@title="Go to page {i+2}"]'
            try:
                next_button = self.driver.find_element(By.XPATH, next_button_xpath)
                next_button.click()
            except NoSuchElementException:
                break
            
            time.sleep(5)
            i += 1
        
        # Save all links and stats to a CSV file
        with open('stack_overflow_links.csv', 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(['S.No', 'Question', 'Votes', 'Answers', 'Views', 'Tags'])
            for i, link in enumerate(all_links):
                writer.writerow([i+1, link, all_stats[k], all_stats[k+1], all_stats[k+2], tag_list[i]])
                k += 3
                
    
    def close_browser(self):
        # Close browser window
        self.driver.quit()

def main():
    stack_overflow = StackOverflowLogin()
    stack_overflow.login('<Mail-Adresss>', '<Password>')
    stack_overflow.get_links_and_stats()

if __name__ == "__main__":
    main()